In [1]:
import pyexasol
import pandas as pd

In [2]:
ip = "localhost:8563"
C = pyexasol.connect(dsn=ip, user='sys', password='exasol', protocol_version=pyexasol.PROTOCOL_V1)

In [3]:
# https://simplemaps.com/data/us-cities
cities = pd.read_csv('uscities.csv')

In [9]:
# nur groesste Stadte, sonst zu viel
cities = cities[cities['ranking'] == 1]

### Table for cities
brauche name, county name, population, id for zip

In [6]:
C.execute(
    """
    CREATE TABLE AOL_SCHEMA.CITY(
        ID int IDENTITY PRIMARY KEY,
        NAME VARCHAR(100),
        STATE_NAME VARCHAR(100),
        COUNTY VARCHAR(100),
        POPULATION int
    );
    """
)

<ExaStatement session_id=1748844784110891673 stmt_idx=3>

### Table for zip codes
soll alle zips zu citiy halten

In [7]:
C.execute(
    """
    CREATE TABLE AOL_SCHEMA.ZIP_CODE(
        ID int CONSTRAINT CITY_ID REFERENCES AOL_SCHEMA.CITY (ID) DISABLE,
        ZIP_CODE VARCHAR(6)
    );
    """
)

<ExaStatement session_id=1748844784110891673 stmt_idx=4>

In [10]:
for i, row in cities.iterrows():
    name = row['city_ascii']
    state = row['state_name']
    county = row['county_name']
    population = row['population']
    
    if "'" in name:
        continue
    if "'" in state:
        continue
    if "'" in county:
        continue
    
    insert = """
                INSERT INTO AOL_SCHEMA.CITY(NAME, STATE_NAME, COUNTY, POPULATION) VALUES
                ('{}', '{}', '{}', '{}');
            """.format(name, state, county, population)
    C.execute(insert)

In [32]:
cities_df = C.export_to_pandas("SELECT * FROM AOL_SCHEMA.CITY")

for i, row in cities_df.iterrows():
    city = row['NAME']
    id_ = row['ID']
    
    i = cities['city_ascii'] == 'New York'
    zips = cities.loc[i, 'zips']
    zips = list(zips)[0].split()
    
    for zip_ in zips:
        insert = """
                INSERT INTO AOL_SCHEMA.ZIP_CODE(ID, ZIP_CODE) VALUES
                ('{}', '{}');
                """.format(id_, zip_)
        C.execute(insert)